## Module 2: The PyTorch Workflow

### Loss Functions

MSELoss - Mean Squared Error (Regression) - Good for predicting continous values (Tempreature)
* Real -> Predicted -> Difference -> Difference squared
* Makes sure all mistakes count
* Punishes bigger errors more than smaller ones.

Cross Entropy Loss (Classification) - predicting a category
* punishes over confident wrong answers.

### MNIST

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
# Data PreProcessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [ ]:
# Load MNIST DataSet
train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform)

In [ ]:
# Create Data Loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [ ]:
# Create the Neural Network
# MNIST image tensors are [1, 28, 28] 1=greyscale, 28=height, 28=width, we add batchsize=64 making it [64, 1, 28, 28]

class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # Flatten this to 28*28 = 784 so its now [64, 784]
        self.layers = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 10) # 10 output classes
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.layers(x)
        return x

In [ ]:
# Device and Optimizer

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}')

# Initialize model and move to device
model = MNISTClassifier().to(device)

# Loss Function and Optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Using cuda


## Traning Loop

In [ ]:
def train_epoch(model, train_loader, loss_function, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        # Track Progress
        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        # Print every 100 batches
        if batch_idx % 100 == 0 and batch_idx > 0:
            avg_loss = running_loss / 100
            accuracy = 100. * correct / total

            print(f'[{batch_idx * 64}/{60000}] '
                  f'Loss: {avg_loss:.3f} | Accuracy: {accuracy:.1f}')
            running_loss = 0.0

In [ ]:
## Evaluation

def evaluate(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return 100. * correct / total

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    print(f'\nEpoch {epoch+1}')
    train_epoch(model, train_loader, loss_function, optimizer, device)
    accuracy = evaluate(model, test_loader, device)
    print(f'Test Accuracy:{accuracy:.2f}%')


Epoch 1
[6400/60000] Loss: 0.645 | Accuracy: 82.0
[12800/60000] Loss: 0.310 | Accuracy: 86.3
[19200/60000] Loss: 0.279 | Accuracy: 88.1
[25600/60000] Loss: 0.243 | Accuracy: 89.3
[32000/60000] Loss: 0.221 | Accuracy: 90.1
[38400/60000] Loss: 0.192 | Accuracy: 90.8
[44800/60000] Loss: 0.180 | Accuracy: 91.4
[51200/60000] Loss: 0.166 | Accuracy: 91.8
[57600/60000] Loss: 0.159 | Accuracy: 92.3
Test Accuracy:94.98%

Epoch 2
[6400/60000] Loss: 0.134 | Accuracy: 95.8
[12800/60000] Loss: 0.127 | Accuracy: 96.1
[19200/60000] Loss: 0.114 | Accuracy: 96.3
[25600/60000] Loss: 0.114 | Accuracy: 96.3
[32000/60000] Loss: 0.104 | Accuracy: 96.4
[38400/60000] Loss: 0.103 | Accuracy: 96.5
[44800/60000] Loss: 0.109 | Accuracy: 96.5
[51200/60000] Loss: 0.115 | Accuracy: 96.5
[57600/60000] Loss: 0.106 | Accuracy: 96.5
Test Accuracy:96.69%

Epoch 3
[6400/60000] Loss: 0.071 | Accuracy: 97.6
[12800/60000] Loss: 0.075 | Accuracy: 97.6
[19200/60000] Loss: 0.085 | Accuracy: 97.5
[25600/60000] Loss: 0.077 | Acc